# Comparative Analysis of Optimization Algorithms 

In  this python script, we will be assesing the performance of algorithms: Particle Swarm Optimisation(PSO) and Genetic Algorithm(GA) in determining the global optima of 3 mathematical benchmark functions: 
1. Shifted Sphere Function
2. Shifted Rastrigin’s Function
3. Shifted Rotated Weierstrass Function

## 1. Particle Swarm Optimisation (PSO) 

Particle Swarm Optimization (PSO),developed by James Kennedy and Russell Eberhart in 1995, is a population-based stochastic 
optimization technique inspired by the social behavior of birds flocking or fish schooling. In PSO, a group of candidate solutions, called particles, navigate the search space by adjusting their positions based on their own experiences and the experiences of neighboring particles. The goal is to find the optimal solution to a problem by iteratively improving the particles' positions according to a fitness function.

### 1.1 Outline of the PSO Algorithm

**Initialization:**

* A swarm of particles is initialized with random positions and velocities within the search space. Each particle represents a potential solution to the optimization problem.
* Each particle’s position is evaluated using the fitness function (the objective function to be minimized or maximized).
* The best position that each particle has found so far (best_local) is recorded.
* Particles are divided into groups, or "informants," which share information about their best-known positions.

**Updating Velocities and Positions:**

In each iteration, the velocity of each particle is updated based on three components:
* Inertia (α): Encourages particles to continue moving in their current direction.
* Cognitive Component (β): Draws particles towards their own best-known position (best_local).
* Social Component (γ): Pulls particles towards the best-known position of their informants (best_informant).
* Global Component (δ): Attracts particles towards the best position found by the entire swarm (best_global).
Each particle’s position is then updated according to its new velocity.

**Boundary Handling:**

* After updating, particles might exceed the problem’s boundaries. If this occurs, their positions are reset randomly within the allowed range.

**Fitness Evaluation:**

* The fitness of each particle’s new position is evaluated.
* If a particle’s new position is better than its previous best (best_local), the position is updated.
* The best position found by each group of informants (best_informant) and by the entire swarm (best_global) are also updated accordingly.

**Termination:**

* The algorithm iterates through the updating process for a predefined number of iterations or until convergence criteria are met (e.g., no significant improvement in fitness).
* The best solution found by the swarm is returned as the optimal solution.

#### Getting Started: Loading Libraries

In [7]:
import numpy as np
import pandas as pd
import optproblems
from optproblems import cec2005
from numpy.random import randint

### 1.2 Explanation of Parameters

* **function** The objective function that the PSO algorithm aims to minimize. Each particle's fitness is evaluated using this 
function.

* **lowerbound** and **upperbound**: Define the boundaries of the search space in each dimension. Particles are initialized 
within this range, and any positions that exceed these boundaries during iterations are reset.

* **num_dimensions**: The dimensionality of the search space. It corresponds to the number of variables in the optimization 
problem.

* **num_particles**: The number of particles in the swarm. A larger swarm can explore more of the search space but may require 
more computational resources.

* **num_informants**: The number of particles in each informant group. Informants share their best-known positions, influencing 
each other’s movement.

* **iterations**: The number of iterations the algorithm runs. More iterations allow particles to refine their positions further, 
but this increases computation time.

* **alpha(α)**: The inertia weight, which controls how much of the previous velocity is retained. A larger value encourages 
exploration, while a smaller value promotes exploitation.

* **beta(β)**: The cognitive component coefficient, which dictates how strongly a particle is pulled towards its own best-known 
position (best_local).

* **gamma(γ)**: The social component coefficient, which determines how strongly a particle is attracted to the best-known position 
of its informants (best_informant).

* **delta(δ)**: The global component coefficient, which influences how much the global best position (best_global) affects each 
particle’s movement.

* **epsilon(ε)**: A damping factor applied to the updated velocity before it’s added to the current position, controlling the 
step size of the particle's movement. This can help stabilize convergence.

By tuning these parameters, you can balance exploration and exploitation, improve convergence speed, and enhance the algorithm’s
ability to escape local optima.

In [8]:
#Some of the ideas for developing this function has been refernced from the code given in the webpage mentioned below 
#webpage: https://python.plainenglish.io/implementing-particle-swarm-optimization-from-scratch-191bc93e48d4

def PSO(function, lowerbound, upperbound, num_dimensions, num_particles, num_informants, iterations, alpha, beta, gamma,delta,epsilon):
    
    swarm_current = []
    swarm_velocity = []

    swarmsize = num_particles
    informants = num_informants
    dimension = num_dimensions
    
    #random initialization of particles
    swarm_current = np.random.uniform(low = lowerbound, high = upperbound, size=(swarmsize,dimension))
    best_local = swarm_current
      
    min_local = np.zeros(swarmsize)
    for i in range(swarmsize):
        min_local[i] = function(swarm_current[i])
      
    best_informant = np.zeros((swarmsize,dimension))
    for i in range(int(swarmsize/informants)):
        for j in range(informants):
            index = i*informants+min_local[i*informants:i*informants+informants].argmin()
            best_informant[informants*i+j] = best_local[index,:]
    
    best_global = best_local[min_local.argmin(),:]
    min_global = min_local.min()
   
    swarm_velocity = np.zeros((swarmsize,dimension))
    for i in range(iterations):        
        
        b = np.random.uniform(low = 0, high = beta, size = (swarmsize,dimension)) #cognitive component
        c = np.random.uniform(low = 0, high = gamma, size = (swarmsize,dimension))
        d = np.random.uniform(low = 0, high = delta, size = (swarmsize,dimension))
        swarm_velocity = alpha*swarm_velocity + b*(best_local - swarm_current) + c*(best_informant - swarm_current)+d*(best_global - swarm_current)
        swarm_current = swarm_current + epsilon*swarm_velocity
        
        for j in range(swarmsize):
            for k in range(dimension):
                if(swarm_current[j,k]<lowerbound or swarm_current[j,k]>upperbound):
                    swarm_current[j,k]=np.random.uniform(low = lowerbound, high = upperbound) 
                
        min = np.zeros(swarmsize)
        for j in range(swarmsize):
             min[j] = function(swarm_current[j])
             
        best_local[(min_local > min),:] = swarm_current[(min_local > min),:]
        min_local = np.minimum(min_local,min)

        for j in range(int(swarmsize/informants)):
            for k in range(informants):
                index = j*informants+min_local[j*informants:j*informants+informants].argmin()
                best_informant[informants*j+k] = best_local[index,:]
    
        min_local = np.minimum(min_local,min)
        best_global = best_local[min_local.argmin(),:]
        min_global = min_local.min()
    
    return min_global         

### 1.3 Investigating using the PSO Function

The investigation_PSO function is designed to systematically investigate the performance of the Particle Swarm Optimization (PSO) algorithm to find the global optima of benchmark functions, by experimenting with various hyperparameter settings. It does so by reading hyperparameters from a csv file, applying these parameters on the PSO algorithm and then recording the results. The end goal is to identify the set of hyperparameters that yields the best global minima for the chosen benchmark function.This approach is crucial for fine-tuning the PSO algorithm to achieve better optimization results in practice.

**Parameters:**

* *filename:* The name of the CSV file containing the hyperparameter configurations.
* *fn_no:* The number identifying which of the 3 mathematical benchmark function to use.

**Returns:**

* A modified DataFrame that includes the mean minima, standard deviation of minima, and the minimum value found for each set of hyperparameters.

In [9]:
def investigation_PSO(filename,fn_no):
    
    data_pso = pd.read_csv(filename)
    pso_mean= []
    pso_std = []
    pso_min= []
    
    for i in range(data_pso.shape[0]):
        
        num_dimensions1 = data_pso.iloc[i,2]
        if(fn_no == 1):
            pso_fn = optproblems.cec2005.F1(num_dimensions1)
        elif(fn_no == 9):
            pso_fn = optproblems.cec2005.F9(num_dimensions1)
        elif(fn_no == 11):
            pso_fn = optproblems.cec2005.F11(num_dimensions1)
        
        lbound1 = data_pso.iloc[i,0]
        ubound1 = data_pso.iloc[i,1]
        num_particles = data_pso.iloc[i,3]
        num_informants = data_pso.iloc[i,4]
        num_iterations1 = data_pso.iloc[i,5]
        alpha_value = data_pso.iloc[i,6]
        beta_value = data_pso.iloc[i,7]
        gamma_value = data_pso.iloc[i,8]
        delta_value = data_pso.iloc[i,9]
        epsilon_value = data_pso.iloc[i,10]
        minima_set1 = []
        
        for j in range(10):
            minima1 = PSO(pso_fn,lbound1,ubound1,num_dimensions1,num_particles,num_informants,num_iterations1,alpha_value,beta_value,gamma_value,delta_value,epsilon_value)
            minima_set1.append(minima1)

        minima_df1 = pd.DataFrame(minima_set1)
        mean_minima1 = np.around(minima_df1[0].mean(),decimals=2)
        std_minima1 = np.around(minima_df1[0].std(),decimals=2)
        min_minima1 = np.around(minima_df1[0].min(),decimals=2)
        pso_mean.append(mean_minima1)
        pso_std.append(std_minima1)
        pso_min.append(min_minima1)
        
    mean_df1 = pd.DataFrame(pso_mean)
    std_df1 = pd.DataFrame(pso_std)
    min_df1 = pd.DataFrame(pso_min)
    
    data_pso[['Mean Minima']] = mean_df1
    data_pso[['Standard Deviation Minima']] = std_df1
    data_pso[['Minimum']] = min_df1

    return data_pso

## 2. Genetic Algorithm(GA)

A Genetic Algorithm (GA) is a popular optimization technique inspired by the principles of natural selection and genetics. It operates by evolving a population of potential solutions over several generations, using mechanisms such as selection, crossover, and mutation to mimic the process of natural evolution. The fittest individuals—those that offer the best solutions according to a fitness function—are selected to produce offspring for the next generation. Over time, this iterative process of selection and variation enables the algorithm to explore the search space and converge on an optimal or near-optimal solution. GAs are particularly effective for solving complex, multi-modal, or discrete optimization problems where traditional methods may struggle.

### 2.1 Outline of GA

**Initialization**

* A population of candidate solutions is randomly generated within the specified bounds. Each candidate solution is represented as a vector in the defined number of dimensions.

* The fitness of each candidate in the population is evaluated using the given benchmark function. The best candidate (with the lowest fitness value) is identified as the global best solution.

**Selection**

* *Tournament Selection:* A subset of the population (tournament) is selected randomly, and the candidate with the best fitness in each tournament is chosen as a parent. This process is repeated to select a set of parents for generating the next generation.

**Crossover**

* One-Point Crossover: Pairs of parents are selected, and for each pair, a crossover point is randomly chosen. The child solutions are created by combining segments of the parent vectors before and after the crossover point.

**Mutation**

* After crossover, a mutation step is applied to the child solutions with a certain probability (mutation rate). This involves randomly altering one or more elements of the child vector to introduce variability and prevent premature convergence.

**Replacement**

* A portion of the existing population is replaced by the newly created children. The selection of which part of the population to replace is random.
* The fitness of the new population is evaluated, and the best solutions are updated accordingly.

**Iteration**
* The algorithm iterates over a defined number of generations, repeating the selection, crossover, mutation, and replacement steps.
* The global best solution is updated if a better solution is found during the iterations.

**Termination**

* After completing the specified number of generations, the algorithm returns the best global solution found during the search process.

This outline closely follows the flow and operations performed in the provided GA code, emphasizing how the genetic algorithm evolves the population of solutions to find an optimal or near-optimal solution.

### 2.2 Explanation of Parameters


* **benchmark**: The objective function that the GA aims to minimize. Each candidate solution’s fitness is evaluated using this function.
* **lowerbound** and **upperbound**: Define the boundaries of the search space. Candidate solutions are initialized within this range and adjusted if they exceed these bounds.
* **num_dimensions**: The number of variables in the optimization problem, representing the dimensionality of the search space.
* **population_size**: The number of candidate solutions in the population. A larger population can explore more of the search space but requires more computational resources.
* **parent_size**: The number of parents selected to create offspring. Determines how many solutions contribute to the next generation.
* **tournament_size**: The number of candidates participating in each tournament selection round. Influences the selection pressure for choosing parents.
* **iterations**: The number of generations the algorithm runs. More iterations provide more opportunities for the population to evolve but increase computation time.
* **mutation_rate**: The probability of randomly altering genes in offspring solutions. Introduces genetic diversity to prevent premature convergence.

By tuning these parameters, you can effectively balance exploration and exploitation, adjust the diversity of the population, and influence the convergence speed of the GA. Properly configuring the population size, tournament size, and mutation rate helps the algorithm to efficiently explore the search space, refine solutions, and avoid premature convergence to local optima.

In [10]:
def GA(benchmark,lowerbound,upperbound,num_dimensions,population_size,parent_size,tournament_size,iterations,mutation_rate):
    
    function = benchmark
    pop_size = population_size
    dimension = num_dimensions
    tourn_size = tournament_size
    par_size = parent_size
    generations = iterations
    
    #creating a population set
    population = np.random.uniform(low = lowerbound,high = upperbound,size = (pop_size,dimension))
    min_population = np.zeros(pop_size)
    for i in range(pop_size):
        min_population[i] = function(population[i])
    
    best_population = population
    best_global = population[min_population.argmin()]
    min_global  = min_population.min()
    
    for k in range(generations):
        
        #tournament selection
        parent = np.zeros((par_size,dimension))
        for i in range(par_size):
            min_index = randint(0,pop_size)
            for j in range(tourn_size):
                index = randint(0,pop_size)
                if(function(population[index]) < function(population[min_index])):
                    min_index = index
            parent[i] =  population[min_index]

        #one-point crossover and mutation
        child = np.zeros((par_size,dimension))
        for i in range(int(par_size/2)):
            parent_A = parent[randint(0,par_size)]
            parent_B = parent[randint(0,par_size)]
            crossover_point = randint(0,dimension+1)
            child_A  = np.concatenate((parent_A[0:crossover_point],parent_B[crossover_point:dimension]))
            child_B  = np.concatenate((parent_B[0:crossover_point],parent_A[crossover_point:dimension]))
            if(np.random.uniform(0,1) < mutation_rate):
                mutation_point_A = randint(0,dimension)
                mutation_point_B = randint(0,dimension)
                child_A[mutation_point_A] = np.random.uniform(lowerbound,upperbound)
                child_B[mutation_point_B] = np.random.uniform(lowerbound,upperbound)
            child[2*i] = child_A
            child[2*i+1] = child_B
            
        index = randint(0,pop_size-par_size)
        population[index:index+par_size] = child
        min = np.zeros(pop_size)
        for i in range(pop_size):
            min[i] = function(population[i])
        best_population[(min_population>min),:] = population[(min_population>min),:]
        population = best_population
        min_population = np.minimum(min_population,min)

        if(min_global > min_population.min()):
            best_global = best_population[min_population.argmin()]
            min_global = min_population.min()
        
    return min_global

### 2.3 Investigating using the GA function

The investigation_GA function is designed to systematically evaluate the performance of the Genetic Algorithm (GA) for finding the global optima of benchmark functions by testing various hyperparameter settings. It reads hyperparameter configurations from a CSV file, applies these configurations to the GA algorithm, and records the resulting performance metrics. The objective is to identify the hyperparameter settings that lead to the best global minima for the selected benchmark function. This process is essential for optimizing the GA algorithm to achieve superior results in practical applications.

**Parameters:**

* *filename:* The name of the CSV file containing the hyperparameter configurations.
* *fn_no:* The number identifying which of the benchmark functions to use.

**Returns:**

A modified DataFrame that includes the mean minima, standard deviation of minima, and the minimum value found for each set of hyperparameters.

In [22]:
def investigation_GA(filename,fn_no):

    data_ga = pd.read_csv(filename)
    ga_mean = []
    ga_std = []
    ga_min = []

    for i in range(data_ga.shape[0]):
        
        num_dimensions2 = data_ga.iloc[i,2]
        if(fn_no == 1):
            ga_fn = optproblems.cec2005.F1(num_dimensions2)
        elif(fn_no == 9):
            ga_fn = optproblems.cec2005.F9(num_dimensions2)
        elif(fn_no == 11):
            ga_fn = optproblems.cec2005.F11(num_dimensions2)
        lbound2 = data_ga.iloc[i,0]
        ubound2 = data_ga.iloc[i,1]
        population_size = data_ga.iloc[i,3]
        parent_size = data_ga.iloc[i,4]
        tournament_size = data_ga.iloc[i,5]
        num_iterations2 = data_ga.iloc[i,6]
        mutation_rate = data_ga.iloc[i,7]
        minima_set2 = []

        for j in range(10):
            minima2 = GA(ga_fn,lbound2,ubound2,num_dimensions2,population_size,parent_size,tournament_size,num_iterations2,mutation_rate)
            minima_set2.append(minima2)
   
        minima_df2 = pd.DataFrame(minima_set2)
        mean_minima2 = np.around(minima_df2[0].mean(),decimals=2)
        std_minima2 = np.around(minima_df2[0].std(),decimals=2)
        min_minima2 = np.around(minima_df2[0].min(),decimals=2)
        ga_mean.append(mean_minima2)
        ga_std.append(std_minima2)
        ga_min.append(min_minima2)
    
    mean_df2 = pd.DataFrame(ga_mean)
    std_df2 = pd.DataFrame(ga_std)
    min_df2 = pd.DataFrame(ga_min)
    data_ga[['Mean Minima']] = mean_df2
    data_ga[['Standard Deviation Minima']] = std_df2
    data_ga[['Minimum']] = min_df2
        
    return data_ga

## 3. Assessing the performance of PSO and GA using the Shifted Sphere Function

The CEC'05 Shifted Sphere function is a well-known benchmark function used for evaluating the performance of optimization algorithms. It is a modified version of the standard Sphere function, one of the simplest and most widely used test functions in optimization.The Shifted Sphere function is mathematically defined as:

$$𝑓(𝑥)=\sum^n_{i=1}(x_i-o_i)^2+f_{bias}$$

where: 
- $x=(x_1,x_2,\dots,x_n)$ is the position vector in an n-dimensional space. 
- $o=(o_1,o_2,\dots,o_n)$ is a shift vector, representing the translation of the function's optimum away from the origin.
- $f_{bias}$ is a constant added to the function to adjust the global minimum value.

In the context of the CEC'05 Shifted Sphere function:
* the range of each $x_i\in[-100,100]$
* $f_{bias}=-450$
* The shifted global optimum o is the vector where the function achieves its global minimum.
* The global optimum value of the function is $f(o)=f_{bias}$.

I have chosen the CEC'05 Shifted Sphere function because its fitness landscape is relatively easy to navigate, which makes it straightforward for algorithms to identify trends and converge towards a solution. However, the function's large domain introduces an additional layer of complexity, requiring algorithms to effectively search over a vast space. This characteristic makes the Shifted Sphere function ideal for evaluating the robustness and accuracy of optimization algorithms, particularly in handling simple yet non-trivial landscapes. The shift away from the origin ensures that algorithms cannot rely on the assumption that the global minimum is at the origin, thereby testing their ability to explore the entire search space thoroughly.

In [12]:
#Reading the csv file with parameters to find global optima of Shifted Sphere Function using PSO algorithm
pso_f1 = investigation_PSO("PSO F1.csv",1)

#Recording the datasets as a csv file
pso_f1.to_csv("F1 Global Optima PSO.csv")

#Printing the best results
pso_f1.nsmallest(5,'Mean Minima')

,Lower Bound,Upper Bound,Number of Dimensions,Swarm size,Number of Informants,Iterations,Alpha,Beta,Gamma,Delta,Epsilon,Mean Minima,Standard Deviation Minima,Minimum
50,-100,100,10,100,10,300,0.5,1.3,1.3,1.4,0.8,-450.00,0.01,-450.0
24,-100,100,10,100,10,100,0.5,1.3,1.3,1.4,0.5,-449.98,0.05,-450.0
48,-100,100,10,100,10,300,0.5,1.3,1.3,1.4,0.5,-449.98,0.06,-450.0
52,-100,100,10,100,10,300,0.5,1.0,1.4,1.6,0.5,-449.98,0.04,-450.0
54,-100,100,10,100,10,300,0.5,1.0,1.4,1.6,0.8,-449.92,0.23,-450.0


In [13]:
#Reading the csv file with parameters to find global optima of Shifted Sphere Function using GA
ga_f1 = investigation_GA("GA F1.csv",1)

#Recording the datasets as a csv file
ga_f1.to_csv("F1 Global Optima GA.csv")

#Printing the best results
ga_f1.nsmallest(5,'Mean Minima')

,Lower Bound,Upper Bound,Number of Dimensions,Population size,Parent size,Tournament size,Iterations,Mutation Rate,Mean Minima,Standard Deviation Minima,Minimum
22,-100,100,10,100,40,10,300,0.4,-448.55,0.82,-449.55
20,-100,100,10,100,40,5,300,0.4,-448.07,0.81,-449.44
23,-100,100,10,100,40,10,300,0.8,-448.00,1.08,-449.33
14,-100,100,10,100,40,10,100,0.4,-436.12,4.90,-443.32
15,-100,100,10,100,40,10,100,0.8,-427.14,10.21,-439.96


### 3.1 Key Observations

Particle Swarm Optimization (PSO) consistently achieves the global minimum of -450.0 for the Shifted Sphere function, with mean minima very close to -450 and minimal standard deviation (0.01 to 0.23), indicating high accuracy and stability. In contrast, Genetic Algorithm (GA) has a best minimum of -449.55 and mean minima ranging from -448.00 to -427.14, with a significantly higher standard deviation (0.81 to 10.21), reflecting greater variability and less reliability. Overall, PSO outperforms GA in both accuracy and consistency for the Shifted Sphere function, making it a more effective optimization method.

## 4. Assessing the performance of PSO and GA using the Shifted Rastrigin’s Function

The CEC'05 Shifted Rastrigin's Function is a well-known benchmark function designed for evaluating optimization algorithms. It is a variant of the standard Rastrigin's Function, a widely used test function known for its multi-modal and complex landscape, which is often used to assess the performance of optimization algorithms.The Shifted Rastrigin's Function is mathematically defined as:

$$f(x)=\sum_{i=1}^n[(x_i-o_i)^2-10\cos(2\pi(x_i-o_i))+10]+f_{bias}$$
where:
- $x=(x_1,x_2,\dots,x_n)$ is a shift vector that translates the function's optimum away from the origin.
- $o=(o_1,o_2,\dots,o_n)$is a constant added to adjust the global minimum value of the function.
- $f_{bias}$ is a constant added to the function to adjust the global minimum value.

In the context of the CEC'05 Shifted Rastrigin's function:
- the range of each  $𝑥_𝑖\in[−5,5]$
- $𝑓_{𝑏𝑖𝑎𝑠}=−330$
- The shifted global optimum o is the vector where the function achieves its global minimum.

I have selected the CEC'05 Shifted Rastrigin's Function due to its challenging fitness landscape. The function’s numerous local minima make it a robust test for optimization algorithms, as they must not only find the global optimum but also effectively navigate a complex, multi-modal space. The shift ensures that algorithms cannot assume the global optimum is at the origin, thus testing their ability to explore and optimize across a broad and intricate search space. This function is ideal for evaluating the robustness and efficiency of optimization algorithms in handling complex, non-trivial landscapes, and for ensuring they can perform well under challenging conditions.

In [21]:
#Reading the csv file with parameters to find global optima of Shifted Rastrigin’s Function using PSO algorithm
pso_f2 = investigation_PSO("PSO F9.csv",9)
pso_f2.to_csv("F9 Global Optima PSO.csv")
pso_f2.nsmallest(5,'Mean Minima')

,Lower Bound,Upper Bound,Number of Dimensions,Swarm size,Number of Informants,Iterations,Alpha,Beta,Gamma,Delta,Epsilon,Mean Minima,Standard Deviation Minima,Minimum
26,-5,5,10,100,10,100,0.5,1.3,1.3,1.4,0.8,-324.53,3.22,-328.01
30,-5,5,10,100,10,100,0.5,1.0,1.4,1.6,0.8,-321.14,2.22,-324.03
25,-5,5,10,100,10,100,0.8,1.3,1.3,1.4,0.5,-321.05,3.93,-325.68
42,-5,5,10,100,40,100,0.5,1.0,1.4,1.6,0.8,-319.95,3.91,-325.03
38,-5,5,10,100,40,100,0.5,1.3,1.3,1.4,0.8,-319.65,3.16,-326.02


In [15]:
#Reading the csv file with parameters to find global optima of Shifted Rastrigin’s Function using GA
ga_f2 = investigation_GA("GA F9.csv",9)
ga_f2.to_csv("F9 Global Optima GA.csv")
ga_f2.nsmallest(5,'Mean Minima')

,Lower Bound,Upper Bound,Number of Dimensions,Population size,Parent size,Tournament size,Iterations,Mutation Rate,Mean Minima,Standard Deviation Minima,Minimum
15,-5,5,10,100,40,10,100,0.8,-326.49,1.19,-328.41
14,-5,5,10,100,40,10,100,0.4,-325.87,1.40,-328.25
12,-5,5,10,100,40,5,100,0.4,-325.18,1.21,-327.25
13,-5,5,10,100,40,5,100,0.8,-321.48,1.73,-324.49
7,-5,5,10,30,10,10,100,0.8,-317.95,4.57,-322.97


### 4.1 Key Observations

Particle Swarm Optimization (PSO) consistently achieves mean minima between -319.65 and -324.53, with best values near -328.01, closely approaching the global minimum of -330. Despite some variability in standard deviation, PSO demonstrates good accuracy and stability. In contrast, Genetic Algorithm (GA) reaches mean minima from -317.95 to -326.49 and best values up to -328.41 but exhibits higher variability and less consistent performance due to sensitivity to parameter settings. Overall, PSO outperforms GA in accuracy and reliability for the Rastrigin function, making it a more effective method for achieving near-global minimum values.

## 5. Assessing the performance of PSO and GA using the Shifted Weierstrass Function

The CEC'05 Shifted Weierstrass Function is a challenging benchmark function designed to test the performance of optimization algorithms. It is a modified version of the standard Weierstrass function, which is known for its highly oscillatory and multi-modal nature. This function is used to evaluate how well optimization algorithms handle complex, non-convex landscapes with numerous local minima.The Shifted Weierstrass Function is mathematically defined as:

$$f(x)=\sum_{i=1}^n\Biggl[\sum_{k=0}^{k_{max}}\Bigl(0.5^k\cos\bigl(2\pi a^k(x_i-o_i)\bigr)\Bigr)-\sum_{k=0}^{k_{max}}\Bigl(0.5^k\cos\bigl(2\pi a^k o_i\bigr)\Bigr)\Biggr]+f_{bias}$$

where:
- $x=(x_1,x_2,\dots,x_n)$ is the position vector in an n-dimensional space.
- $o=(o_1,o_2,\dots,o_n)$ is a shift vector that translates the function's optimum away from the origin.
- $𝑓_{𝑏𝑖𝑎𝑠}$ is a constant added to the function to adjust the global minimum value.

In the context of the CEC'05 Shifted Rastrigin's function:
- the range of each  $𝑥_𝑖\in[−0.5,0.5]$
- $𝑓_{𝑏𝑖𝑎𝑠}=90$
- The shifted vector $o$ is randomly generated, shifting the global optimum away from the origin.

I have selected the CEC'05 Shifted Weierstrass Function because of its complex, multi-modal landscape characterized by numerous local minima and oscillations. This function tests the ability of optimization algorithms to navigate through a highly intricate search space and to avoid being trapped in local optima. The shift away from the origin ensures that algorithms cannot rely on the assumption that the global minimum is at the origin, thereby challenging their ability to explore and optimize effectively across the entire search space. This makes the Shifted Weierstrass Function an ideal choice for evaluating the robustness and efficiency of optimization algorithms under complex and non-trivial conditions.

In [20]:
#Reading the csv file with parameters to find global optima of Shifted Weierstrass Function using PSO algorithm
pso_f3 = investigation_PSO("PSO F11.csv",11)
pso_f3.to_csv("F11 Global Optima PSO.csv")
pso_f3.nsmallest(5,'Mean Minima')

,Lower Bound,Upper Bound,Number of Dimensions,Swarm size,Number of Informants,Iterations,Alpha,Beta,Gamma,Delta,Epsilon,Mean Minima,Standard Deviation Minima,Minimum
26,-0.5,0.5,10,100,10,100,0.5,1.3,1.3,1.4,0.8,94.49,2.22,91.25
34,-0.5,0.5,10,100,10,100,0.5,1.0,1.0,2.0,0.8,94.81,2.41,92.37
30,-0.5,0.5,10,100,10,100,0.5,1.0,1.4,1.6,0.8,94.93,2.11,91.51
32,-0.5,0.5,10,100,10,100,0.5,1.0,1.0,2.0,0.5,95.03,1.70,93.18
28,-0.5,0.5,10,100,10,100,0.5,1.0,1.4,1.6,0.5,95.12,1.30,92.74


In [17]:
#Reading the csv file with parameters to find global optima of Shifted Weierstrass Function using GA 
pso_f3 = investigation_PSO("PSO F11.csv",11)
ga_f3 = investigation_GA("GA F11.csv",11)
ga_f3.to_csv("F11 Global Optima GA.csv")
ga_f3.nsmallest(5,'Mean Minima')

,Lower Bound,Upper Bound,Number of Dimensions,Population size,Parent size,Tournament size,Iterations,Mutation Rate,Mean Minima,Standard Deviation Minima,Minimum
15,-0.5,0.5,10,100,40,10,100,0.8,98.08,1.22,97.02
14,-0.5,0.5,10,100,40,10,100,0.4,98.33,1.34,96.46
12,-0.5,0.5,10,100,40,5,100,0.4,98.42,1.37,96.16
13,-0.5,0.5,10,100,40,5,100,0.8,98.51,1.19,96.46
7,-0.5,0.5,10,30,10,10,100,0.8,99.50,1.07,97.49


### 5.1 Key Observations

Particle Swarm Optimization (PSO) shows greater accuracy in approaching the global minimum of 90 for the Shifted Weierstrass function, with mean minima ranging from 94.49 to 95.12 and best values between 91.25 and 93.18. Although the results are not perfect, PSO's low standard deviation indicates stable performance across different settings. In contrast, Genetic Algorithm (GA) yields mean minima between 98.08 and 99.50, with best values between 96.16 and 97.49, falling further from the global optimum. GA demonstrates more consistent results but with less accuracy, showing that PSO is generally more effective in reaching closer to the global minimum.